<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold">
<br>RUNNING FANAL ANALYSIS</p><br>

This notebook illustrates how to run the FANAL analysys departing from NEXUS files.

The result of the FANAL analysis will be the probability of the simulated events to be interpreted as a signal event so, although we typycally refer to it as "*rejection factor*", strictly speaking it is an "*acceptance factor*". It also provides in a Pandas DataFrame format all the information related to "events", "tracks" and "vovels" generated during the analysis.

The process to run the analysis consists in creating a FanalSetup instance with its details (detector, input nexus files, output fils, analysis parameters ...), and the action of running the analysis. This NB illustrates 2 different ways to build the setup and run the analysis:
1. Loading the Setup config parameters from a config file (json formatted).
2. Building the Setup by providing by hand all the parameters needed.

`FANAL analysis` process is run in a per-event basis and basically consists on a series of filters that events must fulfill to pass to the next analysis step. Events passing all the filters are considered as signal events. Following, a summary of the main steps of the analysis is presented:
> * `MC filter`: Checks that MC event energy fits into limits, there is no signal in BUFFER, and there event has a single S1 signal.
> * Reconstructed hits = MC ionization hits with energy and positions smeared; and shifted z_position (if needed by hit time).
> * `energy filter`: Checks if the smeared event energy fits into limits.
> * Voxelize reconstructed hits using IC Paolina package.
> * `fiducial filter`: Checks if there is any voxel in the non-fiducial region with energy higher than threshold.
> * Make tracks from voxels using IC Paolina package.
> * `track filter`: Checks if the number of tracks is lower or equal the number set by parameter. Currently set to 1.
> * Get blobs from tracks using IC Paolina package.
> * `blob filter`: Checks if blob energies are higher than a certain energy threshold set by parameter.
> * `ROY filter`: Checks if the event smeared energy fts into ROI limits set by parameter.

The NB "fanal_results.ipynb" shows the different data stored in FANAL analysis with associated plots.

#### Setting general stuff

In [ ]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
%load_ext memory_profiler
%load_ext line_profiler

import line_profiler
profile = line_profiler.LineProfiler()

#### Importing

In [ ]:
# General importings

import glob
import json

In [ ]:
# Specific IC stuff

import invisible_cities.core.system_of_units  as units

In [ ]:
# Specific FANAL stuff

from fanal.fanal_setup       import Setup
from fanal.core.fanal_types  import AnalysisParams

# RUNNING FANAL

## from a given config file (json format) ...

In [ ]:
config_fnames = glob.glob("../config/*.config")
print ('Available config files:\n')
for config_fname in config_fnames: print(config_fname) 

In [ ]:
#config_fname = "../config/fanal.next100.Bi214.fwhm_05.voxel_3x3x3.config"
#config_fname = "../config/fanal.next100.Bi214.fwhm_07.voxel_10x10x10.config"
#config_fname = "../config/fanal.next100.Tl208.fwhm_05.voxel_3x3x3.config"
#config_fname = "../config/fanal.next100.Tl208.fwhm_07.voxel_10x10x10.config"
#config_fname = "../config/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.config"
config_fname = "../config/fanal.next100.bb0nu.fwhm_07.voxel_10x10x10.config"

fanal_setup = Setup.from_config_file(config_fname)

In [ ]:
%%time

fanal_setup.run_analysis()

## from parameters ...

In [ ]:
# General stuff
det_name        = 'NEXT100'
event_type      = 'bb0nu'
input_fname     = '../data/next100/bb0nu/sim/*.h5'
output_fname    = '../data/next100/bb0nu/fanal/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.h5'
verbosity_level = 'WARNING' # ('DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL')

analysis_params = AnalysisParams(buffer_Eth        = 0.   * units.keV,
                                 trans_diff        = 0.0  * units.mm / units.cm**0.5,
                                 long_diff         = 0.0  * units.mm / units.cm**0.5,
                                 fwhm              = 0.7  * units.perCent,
                                 e_min             = 2.4  * units.MeV,
                                 e_max             = 2.5  * units.MeV,
                                 voxel_size_x      = 10.  * units.mm,
                                 voxel_size_y      = 10.  * units.mm,
                                 voxel_size_z      = 10.  * units.mm,
                                 strict_voxel_size = True,
                                 voxel_Eth         = 0.   * units.keV,
                                 veto_width        = 20.  * units.mm,
                                 veto_Eth          = 10.  * units.keV,
                                 track_Eth         = 0.   * units.keV,
                                 max_num_tracks    = 1,
                                 blob_radius       = 21.  * units.mm,
                                 blob_Eth          = 350  * units.keV,
                                 roi_Emin          = 2454 * units.keV,
                                 roi_Emax          = 2471 * units.keV
                                )


# Executing FANAL
fanal_setup = Setup(det_name        = det_name,
                    event_type      = event_type,
                    input_fname     = input_fname,
                    output_fname    = output_fname,
                    analysis_params = analysis_params,
                    verbosity       = verbosity_level)

In [ ]:
%%time

fanal_setup.run_analysis()

# SHOWING FANAL RESULTS

In [ ]:
config_df  = fanal_setup.config_df()
events_df  = fanal_setup.events_df()
tracks_df  = fanal_setup.tracks_df()
voxels_df  = fanal_setup.voxels_df()
results_df = fanal_setup.results_df()

In [ ]:
events_df.columns

In [ ]:
events_df[['num_mcParts', 'num_mcHits', 'mc_energy', 'sm_energy', 'energy_filter', 'num_voxels',
           'fiduc_filter', 'num_tracks', 'track_length', 'track_filter',
           'blob1_energy', 'blob2_energy', 'blob_filter', 'roi_filter']].head()

In [ ]:
tracks_df.head()

In [ ]:
voxels_df.head()

In [ ]:
results_df

In [ ]:
from fanal.containers.events import EventCounter

print(EventCounter(**results_df.events))